In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
%matplotlib inline

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
data=fetch_california_housing()
X,y=data.data,data.target

In [3]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=42)
sc=StandardScaler()
X_train_s=sc.fit_transform(X_train)
X_test_s=sc.transform(X_test)

In [5]:
initial_learning_rate=0.01
n_decay_step=1000
n_decay_rate=1/10
momentum=0.9
n_h1=10
n_h2=10
n_h3=1
n_batch=64
n_loop=X_train_s.shape[0]//n_batch
n_epoch=10
scale=0.001
dropout_rate=.5

tf.reset_default_graph()

def shuff_data(X,y):
    index = np.random.permutation(X.shape[0])
    return X[index],y[index]

def get_batch(X,y,batch_index,batch_size):
    return X[batch_index*batch_size:(batch_index+1)*batch_size],y[batch_index*batch_size:(batch_index+1)*batch_size]

initlizer = tf.initializers.variance_scaling(mode='fan_avg')
partial_layer=partial(tf.layers.dense,activation=tf.nn.elu,kernel_initializer=initlizer,kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))
    


X=tf.placeholder(tf.float32,shape=(None,X_train_s.shape[1]),name='X')
y=tf.placeholder(tf.float32,shape=(None,),name='y')

training=False
with tf.name_scope('dnn'):
    # h1=tf.layers.dense(X,n_h1,activation=tf.nn.elu,kernel_initializer=initlizer,name='h1')
    # h2=tf.layers.dense(h1,n_h2,activation=tf.nn.elu,kernel_initializer=initlizer,name='h2')
    # h3=tf.layers.dense(h2,n_h3,name='h3')

    X_drop=tf.layers.dropout(X,rate=dropout_rate,training=training)
    h1=partial_layer(X_drop,n_h1,name='h1')
    h1_drop=tf.layers.dropout(h1,rate=dropout_rate,training=training)
    h2=partial_layer(h1,n_h2,name='h2')
    h2_drop=tf.layers.dropout(h2,rate=dropout_rate,training=training)
    h3=partial_layer(h2,n_h3,activation=None,name='h3')

with tf.name_scope('train'):
    err=y-h3
    base_loss = tf.reduce_mean(tf.square(err),name='base_loss')
    regularizaiton_loss=tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss=tf.add_n([base_loss]+regularizaiton_loss,name='loss')
    global_step=tf.Variable(0,trainable=False,name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate,global_step,n_decay_step,n_decay_rate)

    training_obj=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=momentum).minimize(loss,global_step=global_step)

with tf.name_scope('eval'):
    acc=tf.reduce_mean(tf.abs(err),name='acc')

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for e in range(n_epoch):
        X_tmp,y_tmp = shuff_data(X_train_s,y_train)
        for l in range(n_loop):
            X_batch,y_batch=get_batch(X_tmp,y_tmp,l,n_batch)
            sess.run(training_obj,feed_dict={X:X_batch,y:y_batch})
        acc_res=acc.eval(feed_dict={X:X_test_s,y:y_test})
        print("epoch:%d,acc:%f"%(e,acc_res))

epoch:0,acc:0.916267
epoch:1,acc:0.907717
epoch:2,acc:0.915508
epoch:3,acc:0.912587
epoch:4,acc:0.906437
epoch:5,acc:0.905772
epoch:6,acc:0.906584
epoch:7,acc:0.906846
epoch:8,acc:0.906934
epoch:9,acc:0.907237
